# Chapter 3 - notebook level 1

## Save NLIW event data in ENU coordinates

In [1]:
import os
import glob
import numpy as np
import xarray as xr
import natsort

# from d2spike.despike import DataArray
# from wootils.plotnice import vert_stack
# from pIMOS.utils.nortek_signature_utils import beam2inst, inst2earth

import sys
sys.path.insert(0, '../../')
from chp3_despike import full_pipe
from chp3_basefuncs import get_sigfiles
from chp3_fluxfuncs import get_nliw_times

import warnings
warnings.filterwarnings("ignore")

In [2]:
moor = 'L150'
field_trip = 'RS19'

## Specify the events

In [3]:
wav_tx = get_nliw_times()

# Set key data directories
nc_dir = r'/mnt/share/Experiments/rs19/archive_FV00/Signature 1000/'
nc_files = glob.glob(os.path.join(nc_dir, f'*{moor}*Signature*.nc'))

# Files need natural sorting!!!
nc_files = natsort.natsorted(nc_files)
# nc_files

t_bef = np.timedelta64(30,'m')
t_aft = np.timedelta64(180,'m')

sig_fil = get_sigfiles(wav_tx, t_bef, t_aft, nc_files)

In [4]:
# Set some parameters
gf_sig = [2,2]
re1 = 0
re2 = 0
max_z = 0.07
max_t = 600 # milliseconds

In [5]:
save_dir = r'/mnt/c/Users/00099894/Large_Data/RS2019'
save_dir = os.path.join(save_dir, 'NLIW')

In [6]:
# Loop through files
for ixll, (file, wave) in enumerate(zip(sig_fil, wav_tx)):
    print(ixll)
    if ixll>-1:
    
        # Load Signature file
        if len(file) == 2:
            sig_full = os.path.join(nc_dir, file[0])
            ds1 = xr.open_dataset(sig_full)
            s_ix1 = (ds1['time'].values >= wave - t_bef) &\
                    (ds1['time'].values <= wave + t_aft)
            xr.Dataset.close(ds1)

            sig_full = os.path.join(nc_dir, file[1])
            ds2 = xr.open_dataset(sig_full)
            s_ix2 = (ds2['time'].values >= wave - t_bef) &\
                    (ds2['time'].values <= wave + t_aft)

            if (len(ds2.time.isel(time=s_ix2)) > 0):
                print('Concat')
                ds = xr.concat([ds1['echo'].sel(time=s_ix1, beam=5), ds2['echo'].sel(time=s_ix2, beam=5)], dim='time')
                corr = xr.concat([ds1['corr'].sel(time=s_ix1, beam=5), ds2['corr'].sel(time=s_ix2, beam=5)], dim='time')
            else:
                ds = ds1['echo'].sel(time=s_ix1, beam=5)
                corr = ds1['corr'].sel(time=s_ix1, beam=5)
            xr.Dataset.close(ds2)
        else:
            sig_full = os.path.join(nc_dir, file)
            dsf = xr.open_dataset(sig_full)

            ## Extract the event
            s_ix = (dsf['time'].values >= wave - t_bef) &\
                        (dsf['time'].values <= wave + t_aft)
        
            ds = dsf['echo'].sel(time=s_ix, beam=5)
            corr = dsf['corr'].sel(time=s_ix, beam=5)
            xr.Dataset.close(dsf)

        # Copy out the new data array to fill
        w_new = ds.copy()
        w_new = full_pipe(ds.T,\
                          corr.T,\
                          corrflag=1, qc0_val=85.5, gf_sig=gf_sig, re1=re1, re2=re2).T

        # fig, ax = vert_stack(1, hsize=14, vsize=4)
        # w_new[::4,::32].plot(cmap=cm.turbid,\
        #                      vmin=35.0,\
        #                      vmax=np.abs(w_new).max(),\
        #                      ax=ax, center=0, cbar_kwargs={'pad':0.01})
        
        # Run the 2D interpolation
        if np.sum(np.isnan(w_new.values)) > 0:
            print('Running 2D interp')
            w_intCT = w_new.T.floatda.interp_2D().T
            w_intCT = xr.DataArray(data=w_intCT, coords=w_new.coords)
        else:
            w_intCT = w_new.copy()
        
        # fig, ax = vert_stack(1, hsize=14, vsize=4)
        # w_intCT[::4,::32].plot(cmap=cm.turbid,\
        #                        vmin=35.0,\
        #                        vmax=np.abs(w_intCT).max(),\
        #                        ax=ax, center=0, cbar_kwargs={'pad':0.01})

        # Save as netcdf
        w_intCT.to_netcdf(path=os.path.join(save_dir, 'NLIW_' + str(ixll+1) + '_echo.nc'))
        print('Saved successfully')

0
(50400,)
(50400,)
Running 2D interp
Saved successfully
1
(50400,)
Failed to interpolate along height axis
(50400,)
Running 2D interp
Saved successfully
2
(50400,)
(50400,)
Running 2D interp
Saved successfully
3
Concat
(50399,)
(50399,)
Running 2D interp
Saved successfully
4
Concat
(50399,)
Failed to interpolate along height axis
(50399,)
Failed to interpolate along height axis
Running 2D interp
Saved successfully
5
(50400,)
(50400,)
Running 2D interp
Saved successfully
6
(50400,)
(50400,)
Running 2D interp
Saved successfully
7
(50400,)
(50400,)
Running 2D interp
Saved successfully
8
Concat
(50399,)
(50399,)
Running 2D interp
Saved successfully
9
(50400,)
(50400,)
Running 2D interp
Saved successfully
10
(50400,)
(50400,)
Running 2D interp
Saved successfully
